<a href="https://colab.research.google.com/github/Wissance/TaxesCalculator/blob/main/taxes_ru_individual_entrepreneur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ################################ Описание ###################################
# Проект: TaxesRu
# Версия: v.0.1
# Этот проект мы создали для расчета налогов и взносов, которые, в частности, 
# мне лично нужно уплатить при ведении деятельности в качестве
# Индивидуального Предпринимателя (ИП), при этом сейчас (в этой версии) 
# мы рассматриваем следующие условия:
# 1. Отсуствуют наемные работники
# 2. Система налогооблажения - УСН 6% (в будщем будет УСН 15%)
# 3. Срок начала действия патента может несовпадать с графиком начала платежей,
# т.е. часть налогов расчитываются как 6% от доходов
# КОНЕЧНО, довольно много сервисов, которые позволяют делать расчеты, НО я
# НЕ ХОЧУ им платить (Например, банк Точка для учета патента
# требует использование полного пакета услуг, а это около 5000 руб в год,
# поэтому имеет смысл сэкономить). Я НЕ ПРЕТЕНДУЮ на РОЛЬ СПАСИТЕЛЯ ЧЕЛОВЕЧЕСТВА
# и НЕ ПЛАНИРУЮ заменить инструменты такие как Контур, TaxCom, "Мое дело" и др.
# Я лишь хочу СЭКОНОМИТь немного деньжат, и буду рад если мой калькулятор
# кому-то поможет рассчитать его налоги и взносы
# АВТОР проекта (Ушаков Михаил, https://github.com/EvilLord666) не несет НИКАКОЙ
# ГАРАНТИИ, вы используете данный калькулятор на свой СТРАХ и РИСК.
# ОДНАКО я БУДУ ВАМ ОЧЕНЬ ПРИЗНАТЕЛЕН если вы поддержите МОЮ РАБОТУ, например,
# одним из следующих способов:
#     1. Поставить звездочку проекту на Github (https://github.com/Wissance/TaxesCalculator)
#     2. Если вам интересно развитие этого проекта, то заведите задачу на
#        github (https://github.com/Wissance/TaxesCalculator) 
#        и напишите что бы вам хотелось, чтобы я добавил в плане
#        удобств расчета / новый механизм расчета / какие-то аспекты, которые
#        помогут автоматизировать аспекты вашей деятельности

import typing
from datetime import date, datetime
from abc import ABCMeta, abstractmethod
from functools import reduce

# Обязательные Взносы в ФОМС (Фонд обязательного медицинского страхования)
required_medical_insurance = {
    # формат: год, сумма взносов в руб
    2019: 6884,
    2020: 8426
}

# Обязательные Взносы в ПФР (ОПС, Обязательное пенсионное страхование)
required_retirement_insurance = {
    # формат: год, сумма взносов в руб
    2019: 29354,
    2020: 32448
}

income_overcome = {
    2019: 300000,
    2020: 300000
}

income_overcome_percent = {
    2019: 0.01,
    2020: 0.01
}


# Класс представляющий собой движение средств на текущем этапе это: сумма и дата
class MoneyMove(metaclass=ABCMeta):
    def __init__(self, move_date: date, value: float, movement_type: int):
        self.__move_date = move_date
        self.__value = value
        self.__movement_type = movement_type

    @abstractmethod
    def get_value(self) -> float:
        return self.__value

    def get_raw_value(self) -> float:
        return self.__value

    def get_move_date(self) -> date:
        return self.__move_date

    def get_movement_type(self) -> int:
        return self.__movement_type

    # данные класса
    __move_date = None
    __value = 0.0
    __movement_type = 0


# Класс представляющий собой доход (приходную операцию)
class Income(MoneyMove):
    def __init__(self, move_date: date, value: float, movement_type: int):
        super(Income, self).__init__(move_date, value, movement_type)

    def get_value(self) -> float:
        value = self.get_raw_value()
        if value < 0:
            raise ValueError("Доход не может быть отрицательным")
        return value


# Класс представляющий собой расход (приходную операцию)
class Expenditure(MoneyMove):
    def __init__(self, move_date: date, value: float, movement_type: int):
        super(Expenditure, self).__init__(move_date, value, movement_type)

    # Для расходов представляем value в виде отрицательного числа
    # для удобства суммирования значений
    def get_value(self) -> float:
        value = self.get_raw_value()
        if value < 0:
            return value
        return -1 * value

# ##############################################################################

In [ ]:
# ############################ Движок калькулятора #############################
# Код который будет учитывать патент и считать налоги и отчисления на основе
# коллекций объектов MoneyMove, Income и Expenditure
# ##############################################################################


class Taxes:
    def __init__(self, taxes: float, medical_insurance_fees: float,
                 retirement_insurance_fees: float):
        self.__taxes = taxes
        self.__medical_insurance_fees = medical_insurance_fees
        self.__retirement_insurance_fees = retirement_insurance_fees

    def get_taxes(self) -> float:
        return self.__taxes

    def get_medical_insurance_fees(self) -> float:
        return self.__medical_insurance_fees

    def get_retirement_insurance_fees(self) -> float:
        return self.__retirement_insurance_fees

    __taxes = 0.0  # ФНС
    __medical_insurance_fees = 0.0  # ФСС
    __retirement_insurance_fees = 0.0  # ПФР


class Calculator(metaclass=ABCMeta):
    @abstractmethod
    def calculate(self, money_moves: typing.List[MoneyMove], year: int, *args, **kwargs) -> Taxes:
        raise NotImplementedError


# ############################################################################
# Класс для расчета налогов и сумм взносов для УСН 6%
# ############################################################################
class IncomeTaxCalculator(Calculator):
    def __init__(self, tax_percent: float, use_patent: bool, patent_start: typing.Optional[date],
                 patent_potential_income: typing.Optional[float]):
        self.__tax_percent = tax_percent
        self.__use_patent = use_patent
        self.__patent_start = patent_start
        self.__patent_potential_income = patent_potential_income

    def calculate(self, money_moves: typing.List[MoneyMove], year: int, *args, **kwargs) -> Taxes:
        total_base = 0.0
        print(money_moves)
        incomes = list(filter(lambda item: type(item) == Income, money_moves))
        # todo: UMV: Расчитать расходы, которые уходят в качестве платежей (ежеквартальных)
        # и уменьшить сумму налогообложения на эту величину
        # total = reduce(lambda a, b: a + b, incomes)
        for income in incomes:
            # Проверка даты
            if self.__use_patent:
                income_date = income.get_move_date()
                if income_date < self.__patent_start:
                    total_base += income.get_value()
            else:
                total_base += income.get_value()
        income_overcome_value = income_overcome[year]
        additional_retirement_insurance = 0.0

        total_base_for_retirement_fees = total_base
        if self.__use_patent:
            total_base_for_retirement_fees += self.__patent_potential_income
        
        if total_base_for_retirement_fees > income_overcome_value:
            additional_retirement_insurance = (total_base_for_retirement_fees - income_overcome_value) * income_overcome_percent[year]

        medical_fees = required_medical_insurance[year]
        retirement_fees = required_retirement_insurance[year] + additional_retirement_insurance

        taxes = Taxes(total_base * self.__tax_percent, medical_fees, retirement_fees)
        return taxes

    __tax_percent = 0.06
    __patent_start = date(2020, 8, 5)
    __use_patent = True
    __patent_potential_income = 100000.0


In [ ]:
# ################################### Тесты ###################################
# На тестовых данных проверим работу движка калькулятора, в данном случае я
# расчитаю размер еалогов и выплат исходя из первых принципов, будем считать,
# что данные уже загружены в памяти
# ##############################################################################
import unittest


class TestIncomeTaxCalculator(unittest.TestCase):

    def test_calculateWithPatentNoLimitOvercome(self):
        """
            Тестовый сценарий:
               1. Калькулятор настраиваем т.о., чтобы потенциальный доход был
                  меньше превышения (при котором требуется платить 1% от
                  превышения)
               2. Первые поступления на Р/С были до получения патента, поэтому
                  они считаются как 6%, а остальные не учитываются, при этом
                  эти доходы не приводят к превышению лимита
        """
        calculator = IncomeTaxCalculator(0.06, True, date(2020, 6, 1), 10000)
        bank_account_money_moves = [
            Income(date(2020, 4, 15), 200000, 0),
            Income(date(2020, 5, 20), 60000, 0),
            Expenditure(date(2020, 5, 27), 144900, 1),
            Income(date(2020, 6, 15), 200000, 0),
            Income(date(2020, 7, 15), 200000, 0),
            Expenditure(date(2020, 7, 17), 78900, 1),
            Income(date(2020, 8, 15), 200000, 0),
            Income(date(2020, 9, 15), 200000, 0),
            Expenditure(date(2020, 9, 22), 278900, 1),
            Income(date(2020, 10, 15), 200000, 0),
            Income(date(2020, 11, 15), 200000, 0),
            Income(date(2020, 12, 15), 200000, 0)
        ]
        self._run_test_and_check(calculator, bank_account_money_moves, 2020,
                                 15600, 8426, 32448)

    def test_calculateWithPatentAndLimitOvercome(self):
        """
            Тестовый сценарий:
                Формируем доход т.о., чтобы он составлял 400000 до патента + 
                сумма патента 100000, которые и будут учитываться при расчете 1% в ПФР
                
        """
        calculator = IncomeTaxCalculator(0.06, True, date(2020, 6, 1), 100000)
        bank_account_money_moves = [
            Income(date(2020, 4, 15), 200000, 0),
            Income(date(2020, 5, 20), 200000, 0),
            Expenditure(date(2020, 5, 27), 144900, 1),
            Income(date(2020, 6, 15), 200000, 0),
            Income(date(2020, 7, 15), 200000, 0),
            Expenditure(date(2020, 7, 17), 78900, 1),
            Income(date(2020, 8, 15), 200000, 0),
            Income(date(2020, 9, 15), 200000, 0),
            Expenditure(date(2020, 9, 22), 278900, 1),
            Income(date(2020, 10, 15), 200000, 0),
            Income(date(2020, 11, 15), 200000, 0),
            Income(date(2020, 12, 15), 200000, 0)
        ]
        self._run_test_and_check(calculator, bank_account_money_moves, 2020,
                                 24000, 8426, 34448)

    def test_calculateNoPatentAtAll(self):
        """
            Тестовый сценарий:
                Моделируем ситуацию, когда необходимо посчитать сколько мы заплатим
                этим наглым и жадным госорганом вообще без патента (глупая затея)
        """
        calculator = IncomeTaxCalculator(0.06, False, None, None)
        bank_account_money_moves = [
            Income(date(2020, 4, 15), 200000, 0),
            Income(date(2020, 5, 20), 200000, 0),
            Expenditure(date(2020, 5, 27), 144900, 1),
            Income(date(2020, 6, 15), 200000, 0),
            Income(date(2020, 7, 15), 200000, 0),
            Expenditure(date(2020, 7, 17), 78900, 1),
            Income(date(2020, 8, 15), 200000, 0),
            Income(date(2020, 9, 15), 200000, 0),
            Expenditure(date(2020, 9, 22), 278900, 1),
            Income(date(2020, 10, 15), 200000, 0),
            Income(date(2020, 11, 15), 200000, 0),
            Income(date(2020, 12, 15), 200000, 0)
        ]
        self._run_test_and_check(calculator, bank_account_money_moves, 2020,
                                 108000, 8426, 47448)

    def test_caclulateWithPatentAndQuarterTaxFees(self):
        """
            Тестовый сценарий:
                
        """
        pass

    def _run_test_and_check(self, taxes_calculator: Calculator, 
                            bank_account_money_moves: typing.List[MoneyMove], 
                            year: int, taxes_fees: float, 
                            medical_insurance_fees: float, 
                            retirement_insurance_fees: float):
        taxes = taxes_calculator.calculate(bank_account_money_moves, year)
        self.assertEqual(taxes_fees, taxes.get_taxes())
        self.assertEqual(medical_insurance_fees, taxes.get_medical_insurance_fees())
        self.assertEqual(retirement_insurance_fees, taxes.get_retirement_insurance_fees())

unittest.main(argv=['first-arg-is-ignored'], exit=False)

....

[<__main__.Income object at 0x7fddef7eddd8>, <__main__.Income object at 0x7fddef7edc88>, <__main__.Expenditure object at 0x7fddef7edcc0>, <__main__.Income object at 0x7fddef7edbe0>, <__main__.Income object at 0x7fddef7edc18>, <__main__.Expenditure object at 0x7fddef7edac8>, <__main__.Income object at 0x7fddef7edb00>, <__main__.Income object at 0x7fddef7edb38>, <__main__.Expenditure object at 0x7fddef7eda58>, <__main__.Income object at 0x7fddef7c0780>, <__main__.Income object at 0x7fddef7c0908>, <__main__.Income object at 0x7fddef7c0940>]
[<__main__.Income object at 0x7fddef7ede80>, <__main__.Income object at 0x7fddef7edc18>, <__main__.Expenditure object at 0x7fddef7edcc0>, <__main__.Income object at 0x7fddef7edbe0>, <__main__.Income object at 0x7fddef7edc88>, <__main__.Expenditure object at 0x7fddef7edac8>, <__main__.Income object at 0x7fddef7edb00>, <__main__.Income object at 0x7fddef7edb38>, <__main__.Expenditure object at 0x7fddef7eda58>, <__main__.Income object at 0x7fddef7edd30>, 


----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


In [ ]:
# ################################### Расчет ###################################
# Для расчета будем использовать следующий подход:
# Все данные будем хранить в виде tsv - файла / Excel документа на Google диске
# Необходимо определить формат хранения данных
# ##############################################################################

